## Bruno de Moraes
## João Nascimento

Importação das bibliotecas necessárias

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from collections import Counter

Leitura dos dados de treino

In [ ]:
raw_data = pd.read_csv('./datasets/train.csv', sep=',')
raw_data.shape

Removendo a coluna ID, pois ela irá impactar negativamente nos resultados

In [ ]:
raw_data.drop(['id'], axis=1, inplace=True)

Copiando para o df_train

In [ ]:
df_train = raw_data.copy()
df_train.head()

Tentamos remover os registros que possuiam algum dado "-999", porém ao finalizar os testes, isso causou um impacto negativo na porcentagem de precisão

In [ ]:
df_train_sem_999 = df_train.replace([-999], pd.NA)
df_train_sem_999.dropna(inplace=True)
df_train_sem_999.shape

Após o teste anterior, tentamos manter apenas as colunas que tivessem a maior correlação com a coluna y (resultado), porém novamente, pela remoção de muitos dados, a porcentagem de precisão diminuiu consideravelmente.

In [ ]:
df_train_corr = df_train.corr()[['y']]
df_train_corr = df_train_corr[df_train_corr > 0]
df_train_corr.dropna(inplace=True)
df_train_mais_correlacao = df_train[df_train_corr.index.values.tolist()]
df_train_mais_correlacao.head()

Definindo os recursos de X e y, baseados no df_train padrão, já que as tentativas anteriores de tratamento não tiveram sucesso

In [ ]:
X = df_train.loc[:, df_train.columns != 'y']
y = df_train['y']

Balanceando X e y, porém em variaveis diferentes das originais, pois vamos utilizar dos dois modos

In [ ]:
oversample = SMOTE()
X_blc, y_blc = oversample.fit_resample(X,y)

Separando os dados de treino e teste desbalanceados

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.30,
                                                    shuffle=True,
                                                    random_state=609)

Separando os dados de treino e teste balanceados

In [ ]:
X_blc_train, X_blc_test, y_blc_train, y_blc_test = train_test_split(X_blc, y_blc,
                                                    test_size = 0.30,
                                                    shuffle=True,
                                                    random_state=609)

# Modelo 1 - Regressão Logística
## Desbalanceado
Criando o modelo e o treinando com os dados

In [ ]:
clr = LogisticRegression()
clr.fit(X_train, y_train)

Fazendo a predição

In [ ]:
y_pred = clr.predict(X_test)

Printando o resultado da predição

In [ ]:
print(classification_report(y_test, y_pred))

Como podemos analisar, a regressão logística desbalanceado obteve 80% de precisão média (macro avg/precision).
<br>
Isso é um resultado satisfatório.

Gerando uma Matrix de Confusão para facilitar a visualização dos falso-positivos

In [ ]:

ConfusionMatrixDisplay.from_estimator(clr, X_test, y_test, cmap='Blues')

Como podemos ver, o modelo acertou muitos dos clientes que não compraram o produto, prevendo certo, pois o dataset possui muitos registros dessa categoria. Já para os clientes que compraram o produto, o modelo apresentou como Falso, errando a maioria dessa categoria.

## Balanceado
Criando o modelo e o treinando com os dados

In [ ]:
clr_blc = LogisticRegression()
clr_blc.fit(X_blc_train, y_blc_train)

Fazendo a predição

In [ ]:
y_blc_pred = clr_blc.predict(X_blc_test)

Printando o resultado da predição

In [ ]:
print(classification_report(y_blc_test, y_blc_pred))

Como podemos analisar, a regressão logística balanceado obteve 61% de precisão média (macro avg/precision).
<br>
Isso é um resultado mediamente satisfatório.

Gerando uma Matrix de Confusão para facilitar a visualização dos falso-positivos

In [ ]:

ConfusionMatrixDisplay.from_estimator(clr_blc, X_blc_test, y_blc_test, cmap='Blues')

Como podemos ver, o modelo acertou 61% dos casos onde o produto não seria comprado, e 61% dos casos onde ele seria comprado. Comparando com a versão desbalanceada, o modelo conseguiu um resultado mais homogenio, porém ao custo de uma menor precisão nos casos onde o produto não seria comprado.